## <Center>本节课知识点提炼</Center>


<!--  -->

### 本节课重点与难点
* 重点：TensorFlow基本使用
* 难点：Caffe的使用

### 深度学习框架与应用


### 1.Caffe

#### 特点  
* 来源于Berkeley的开源框架
* 高效、一般的训练无需手写大量代码
* 有python和mathlab的接口
* 对于卷积神经网络的训练和fine-tuning非常方便

#### 安装  
* Ubuntu 14.04+的同学请参考手把手教程  [链接](https://www.zybuluo.com/hanxiaoyang/note/364737 )
 
* CentOS 7.0+的同学请参考手把手教程  [链接](https://www.zybuluo.com/hanxiaoyang/note/364680)

* Mac的同学可以参考 [链接](https://www.zybuluo.com/hanxiaoyang/note/370344)

* Ubuntu >=17.04下快速配置Caffe  [教程链接](http://caffe.berkeleyvision.org/install_apt.html)   
    CPU版本:sudo apt install caffe-cpu  
    GPU版本:sudo apt install caffe-cuda
    
#### Caffe官方实践-Training LeNet on MNIST with Caffe [教程链接](http://caffe.berkeleyvision.org/gathered/examples/mnist.html)  

1) 准备数据  
MNIST数据路径： \$ CAFFE_ROOT/data/mnist/     
cd \\$CAFFE_ROOT  
#运行shell命令获取MNIST数据  
./data/mnist/get_mnist.sh    
#将数据转换格式为LMDB数据库格式  
./examples/mnist/create_mnist.sh  
2) 配置网络结构  
官网提供了定义好的网络文件 \$CAFFE_ROOT/examples/mnist/lenet_train_test.prototxt   
内容如下：

In [ ]:
name: "LeNet"  
layer {  
  name: "mnist"  
  type: "Data" #数据层  
  top: "data"   
  top: "label"  
  include {  
    phase: TRAIN #训练时才加载  
  }  
  transform_param {  
    scale: 0.00390625 #每个像素乘以改值做归一化（1/255 = 0.00390625）  
  }  
  data_param {  
    source: "examples/mnist/mnist_train_lmdb" #前面生成的训练数据集  
    batch_size: 64 # 每一批训练集大小  
    backend: LMDB #数据格式  
  }  
}  
layer {  
  name: "mnist"    
  type: "Data" #数据层  
  top: "data"  
  top: "label"  
  include {  
    phase: TEST #测试时才加载  
  }  
  transform_param {  
    scale: 0.00390625  
  }  
  data_param {  
    source: "examples/mnist/mnist_test_lmdb" #前面生成的测试数据集  
    batch_size: 100  
    backend: LMDB  
  }  
}  
layer {  
  name: "conv1"  
  type: "Convolution" #卷积层  
  bottom: "data"  
  top: "conv1"  
  param {  
    lr_mult: 1 #weights学习率  
  }  
  param {  
    lr_mult: 2 #bias学习率  
  }  
  convolution_param {  
    num_output: 20 #输出多少个特征图（对应卷积核数量）  
    kernel_size: 5 #卷积核大小  
    stride: 1 #步长  
    weight_filler {  
      type: "xavier" #权重初始化算法  
    }  
    bias_filler {  
      type: "constant" #基值初始化算法  
    }  
  }  
}  
layer {  
  name: "pool1"  
  type: "Pooling" #池化层  
  bottom: "conv1"   
  top: "pool1"  
  pooling_param {  
    pool: MAX #池化方法  
    kernel_size: 2  
    stride: 2  
  }  
}  
layer {  
  name: "conv2"  
  type: "Convolution"  
  bottom: "pool1"  
  top: "conv2"  
  param {  
    lr_mult: 1  
  }  
  param {  
    lr_mult: 2  
  }  
  convolution_param {  
    num_output: 50  
    kernel_size: 5  
    stride: 1  
    weight_filler {  
      type: "xavier"  
    }  
    bias_filler {  
      type: "constant"  
    }  
  }  
}  
layer {  
  name: "pool2"  
  type: "Pooling"  
  bottom: "conv2"  
  top: "pool2"  
  pooling_param {  
    pool: MAX  
    kernel_size: 2  
    stride: 2  
  }  
}  
layer {  
  name: "ip1"  
  type: "InnerProduct" #全链接层  
  bottom: "pool2"  
  top: "ip1"  
  param {  
    lr_mult: 1 #weights学习率  
  }  
  param {  
    lr_mult: 2 #bias学习率  
  }  
  inner_product_param {  
    num_output: 500  
    weight_filler {  
      type: "xavier"  
    }  
    bias_filler {  
      type: "constant"  
    }  
  }  
}  
layer {  
  name: "relu1"  
  type: "ReLU" #relu层  
  bottom: "ip1"  
  top: "ip1"  
}  
layer {  
  name: "ip2"  
  type: "InnerProduct"  
  bottom: "ip1"  
  top: "ip2"  
  param {  
    lr_mult: 1  
  }  
  param {  
    lr_mult: 2  
  }  
  inner_product_param {  
    num_output: 10  
    weight_filler {  
      type: "xavier"  
    }  
    bias_filler {  
      type: "constant"  
    }  
  }  
}  
layer {  
  name: "accuracy"  
  type: "Accuracy" #输出精度  
  bottom: "ip2"  
  bottom: "label"  
  top: "accuracy"  
  include {  
    phase: TEST  
  }  
}  
layer {  
  name: "loss"   
  type: "SoftmaxWithLoss" #输出损失  
  bottom: "ip2"  
  bottom: "label"  
  top: "loss"  
}  

可以用官方自带的python绘图工具绘制出网络图  
./caffe/python/draw_net.py ./caffe/examples/mnist/lenet_train_test.prototxt ./lenet_train_test.png  
    在当前路径下生成lenet_train_test.png

3)配置网络参数  
官网给出了参数配置文件：\$CAFFE_ROOT/examples/mnist/lenet_solver.prototxt   

    #The train/test net protocol buffer definition  
    net: "examples/mnist/lenet_train_test.prototxt"  
    #test_iter specifies how many forward passes the test should carry out.  
    #In the case of MNIST, we have test batch size 100 and 100 test iterations,  
    #covering the full 10,000 testing images.  
    test_iter: 100  
    #Carry out testing every 500 training iterations. 设置每500次测试一下网络 精度 损失  
    test_interval: 500  
    #The base learning rate, momentum and the weight decay of the network.  
    base_lr: 0.01  
    momentum: 0.9  
    weight_decay: 0.0005  
    #The learning rate policy  
    lr_policy: "inv"  
    gamma: 0.0001  
    power: 0.75   
    #Display every 100 iterations  设置每100次迭代训练显示当前状态 lr loss  
    display: 100  
    #The maximum number of iterations   
    max_iter: 10000  
    #snapshot intermediate results中间结果快照每5000次保存一次  
    snapshot: 5000  
    snapshot_prefix: "examples/mnist/lenet"   
    #solver mode: CPU or GPU  
    solver_mode: GPU  

4)训练  
* 执行shell脚本，开始训练  
./examples/mnist/train_lenet.sh  
训练好的模型保存路径: examples/mnist  
* 对训练结果进行测试：  
./build/tools/caffe test -model examples/mnist/lenet_train_test.prototxt -weights examples/mnist/lenet_iter_10000.caffemodel  -iterations 100

5)用python接口调用训练号的模型识别数字

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
caffe_root = 'your_caffe_path'
sys.path.insert(0, caffe_root + 'python') #把pycaffe所在路径添加到环境变量 
import caffe

MODEL_FILE = os.pathjoin(caffe_root, 'examples/mnist/lenet.prototxt') 
PRETRAINED = os.pathjoin(caffe_root, 'examples/mnist/lenet_iter_10000.caffemodel') 
#图片处理成 lenet.prototxt的输入要求（尺寸28x28）且已经二值化为黑白色
IMAGE_FILE = 'test.bmp'
 
input_image = caffe.io.load_image(IMAGE_FILE, color=False)
net = caffe.Classifier(MODEL_FILE, PRETRAINED) 
prediction = net.predict([input_image], oversample = False)
caffe.set_mode_cpu()
print('predicted class:', prediction[0].argmax())

### 2.TensorFlow

#### 1) TensorFlow的一些概念:  
* 使用张量(tensor)表示数据. 
* 使用图(graph)来表示计算任务. 
* 在被称之为会话(Session)的上下文 (context)中执行图. 
* 通过变量 (Variable)维护状态. 
* 使用feed和fetch可以为任意的操作(arbitrary operation)赋值或者从其中获取数据   

#### 2) 获取张量的值(使用eval属性)：   

    x = tf.zeros((2,2))  
    x.eval()  
    输出：[[0 0] [0 0]]  
    
#### 3) Tensorflow的计算   
  构造部分(计算流图) => 执行部分(通过session来执行图中的计算)
* 定义计算图

In [ ]:
import tensorflow as tf
# 创建一个常量节点， 产生一个1x2矩阵，这个op被作为一个节点
# 加到默认视图中
# 构造器的返回值代表该常量节点的返回值 
matrix1 = tf.constant([[3,3]])
# 创建另一个常量节点，产生一个2x1的矩阵 
matrix2 = tf.constant([2],[2]).
# 创建一个矩阵乘法matmu1节点，把matrix1和matrix2作为输入： 
product = tf.matmul(matrix1, matrix2)

* 通过TF中的session执行上面的操作  

In [ ]:
# 创建session，启动默认图 
sess = tf.Session()  
# 调用sess的'run()'方法来执行矩阵乘法节点操作, 传入'product'作为该方法的参数。'product'代表了矩阵乘法节点的输出，传人它是告诉方法我们希望取回矩阵乘法节点的输出。
# 整个执行过程是自动化的，会话负责传递节点所需的全部输入。节点通常是并发执行的。
# 函数调用'run(product)'会触发图中三个节点（上面提到的两个常量节点和一个矩阵乘法节点）的执行。
# 返回值'result'是一个numpy 'ndarray'对象。
result = sess.run(product) 
print(result)
# 结果为[[12.]]
# 完成任务, 记得关闭会话 
sess.close()

* 用with…device语句用来指派特定的CPU或GPU操作   
with tf.device("/gpu:0"):  
/cpu:0 使用CPU  
/gpu:0 使用编号为0的GPU  
/gpu:1 使用编号为1的GPU

#### 4) TensorBoard 训练可视化  
使用 tensorboard --lodir='./logs'命令将保存的日志文件进行可视化(例如：可视化loss/accuracy等)

#### 5)TensorFlow实践  
参考课件代码

### 3.PyTorch
#### 特点
* 高速计算  
* 自动求导

#### 基本概念  
* torch.Tensor – 类似numpy.array的张量
* autograd.Variable – 对Tensor的封装，可自动求导
* autograd.Function – 对Variables的运算，完成前向(forward )和反向(backward)运算.
* nn.Parameter – 一种特殊的Variable
* nn.Module – 包含参数，以及输入变量上定义的函数

#### 代码实践
参考课件